In [ ]:
#

# 3. ALGORITHMIC QUESTION

You are given a list of integers, A, and another integer s. Write an algorithm that outputs all the pairs in A that equal x.

For example, if

A = [7, -2, 8, 2, 6, 4, -7, 2, 1, 3, -3] and s = 4
the algorithm should output: (7, -3), (-2, 6), (2, 2), (3, 1).